# 🌾 CropVision-AI Advanced Training Pipeline

This notebook implements the advanced training pipeline requested:
1.  **Student Model**: MobileNetV3 Large (with SE Attention).
2.  **Teacher Model**: ResNet50 (Knowledge Distillation).
3.  **Optimization**: Structured Pruning + INT8 Quantization Aware Training (QAT).

**Hardware**: GPU Required (Runtime -> Change runtime type -> T4 GPU).

In [ ]:
# 1. Setup & Check GPU
!nvidia-smi
import torch
print(f"PyTorch Version: {torch.__version__}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 2. Prepare Dataset
Downloading PlantVillage dataset.

In [ ]:
# Download PlantVillage Dataset
!wget -q https://github.com/spMohanty/PlantVillage-Dataset/archive/refs/heads/master.zip -O plantvillage.zip
!unzip -q plantvillage.zip
!mv PlantVillage-Dataset-master/raw/color ./data
!rm -rf PlantVillage-Dataset-master plantvillage.zip

import os
print(f"Dataset classes: {len(os.listdir('./data'))}")

## 3. Define Modules
Defining models, distillation loss, pruning, and quantization logic inline.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
import torch.quantization
from torch.ao.quantization import QuantStub, DeQuantStub
import torch.nn.utils.prune as prune
import copy

# --- 1. Distillation Loss ---
class KnowledgeDistillationLoss(nn.Module):
    def __init__(self, alpha=0.5, temperature=3.0):
        super(KnowledgeDistillationLoss, self).__init__()
        self.alpha = alpha
        self.temperature = temperature
        self.criterion_ce = nn.CrossEntropyLoss()
        self.criterion_kl = nn.KLDivLoss(reduction='batchmean')

    def forward(self, student_logits, teacher_logits, labels):
        teacher_probs = F.softmax(teacher_logits / self.temperature, dim=1)
        student_log_probs = F.log_softmax(student_logits / self.temperature, dim=1)
        loss_kd = self.criterion_kl(student_log_probs, teacher_probs) * (self.temperature ** 2)
        loss_ce = self.criterion_ce(student_logits, labels)
        return self.alpha * loss_kd + (1 - self.alpha) * loss_ce

# --- 2. Model Definitions ---
def get_student_model(num_classes):
    # MobileNetV3 Large (includes SE blocks)
    model = models.mobilenet_v3_large(weights=models.MobileNet_V3_Large_Weights.IMAGENET1K_V1)
    model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)
    return model

def get_teacher_model(num_classes):
    # ResNet50 Teacher
    model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
    model.fc = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(model.fc.in_features, num_classes)
    )
    # In a real scenario, you would load pre-trained teacher weights here.
    # For this script, we assume the teacher learns alongside or is just ImageNet initialized if no path provided.
    # To save time in this demo, we will finetune the teacher briefly first or just use it as is if allowed.
    # Here we will freeze it after initialization to simulate a pre-trained teacher.
    for param in model.parameters():
        param.requires_grad = False
    model.eval()
    return model

# --- 3. Pruning Utils ---
def apply_structured_pruning(model, amount=0.2):
    print(f"Applying structured pruning (amount={amount})...")
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d):
            prune.ln_structured(module, name='weight', amount=amount, n=1, dim=0)

def remove_pruning_reparameterization(model):
    print("Making pruning permanent...")
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d):
            try:
                prune.remove(module, 'weight')
            except ValueError:
                pass

# --- 4. Quantization Utils ---
def prepare_model_for_qat(model):
    print("Preparing model for QAT...")
    # 'qnnpack' is good for mobile/arm, 'fbgemm' for x86 server.
    # On Colab (Linux x86), fbgemm is standard. If errors occur, switch to qnnpack.
    model.qconfig = torch.ao.quantization.get_default_qat_qconfig('fbgemm')
    torch.ao.quantization.prepare_qat(model, inplace=True)
    return model

def convert_qat_model(model):
    print("Converting QAT model to INT8...")
    model.eval()
    return torch.ao.quantization.convert(model, inplace=False)

## 4. Training Loop

In [ ]:
from torch.utils.data import DataLoader, random_split
from torchvision import datasets
import torch.optim as optim

# Config
BATCH_SIZE = 32
EPOCHS = 10  # Increase for better results
LR = 0.001

# Data Loaders
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset = datasets.ImageFolder('./data', transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_data, val_data = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

num_classes = len(dataset.classes)
print(f"Classes: {num_classes}")

def train_one_epoch(model, teacher, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        student_logits = model(images)
        with torch.no_grad():
            teacher_logits = teacher(images)
        loss = criterion(student_logits, teacher_logits, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = student_logits.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    return running_loss/len(loader), 100.*correct/total

def validate(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return 100.*correct/total

In [ ]:
# Setup Models
student = get_student_model(num_classes).to(device)
teacher = get_teacher_model(num_classes).to(device) # In real usage, load weights here!

criterion = KnowledgeDistillationLoss()
optimizer = optim.Adam(student.parameters(), lr=LR)

# Phase 1: Distillation
print("--- Phase 1: Distillation ---")
best_acc = 0.0
for epoch in range(EPOCHS):
    loss, acc = train_one_epoch(student, teacher, train_loader, criterion, optimizer, device)
    val_acc = validate(student, val_loader, device)
    print(f"Epoch {epoch+1} - Loss: {loss:.4f}, Val Acc: {val_acc:.2f}%")
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save({
            'model_state_dict': student.state_dict(),
            'classes': dataset.classes
        }, "student_best.pth")

# Reload best
checkpoint = torch.load("student_best.pth")
student.load_state_dict(checkpoint['model_state_dict'])

# Phase 2: Pruning
print("\n--- Phase 2: Pruning ---")
apply_structured_pruning(student, amount=0.2)
# Fine-tune
optimizer = optim.Adam(student.parameters(), lr=LR * 0.1)
for epoch in range(3): # Short fine-tune
    loss, acc = train_one_epoch(student, teacher, train_loader, criterion, optimizer, device)
    print(f"Pruning FT Epoch {epoch+1} - Loss: {loss:.4f}")
remove_pruning_reparameterization(student)

# Phase 3: QAT
print("\n--- Phase 3: QAT ---")
student.to('cpu')
teacher.to('cpu')
student.train() # IMPORTANT for QAT prepare
student = prepare_model_for_qat(student)
optimizer = optim.Adam(student.parameters(), lr=LR * 0.01)
criterion_ce = nn.CrossEntropyLoss()

for epoch in range(3):
    student.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = student(images)
        loss = criterion_ce(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"QAT Epoch {epoch+1} - Loss: {running_loss/len(train_loader):.4f}")

quantized_model = convert_qat_model(student)

# Save Final Quantized Model (Scripted for portability)
try:
    scripted_model = torch.jit.script(quantized_model)
    torch.jit.save(scripted_model, "quantized_model_scripted.pt")
    print("Saved: quantized_model_scripted.pt")
except Exception as e:
    print(f"Scripting failed: {e}")
    torch.save(quantized_model.state_dict(), "quantized_model_state.pth")

## 5. Download Model
Run this cell to download the final model file.

In [ ]:
from google.colab import files
files.download('quantized_model_scripted.pt')